Задание:

С помощью метода get_prepared_all_vacancies_from_hh
загрузить 10000 вакансии с сайта hh.ru.

Договоритесь чтобы вакансии не повторялись.

Сохранить результат в виде csv файла с тремя столбцами:
1. id вакансии
2. сырой json
3. подготовленное описание вакансии

Не забывайте про обработку ошибок

In [ ]:
import concurrent.futures as cf
import pandas as pd
from urllib.request import urlopen
import json
import re
import nltk.tokenize as nt
import nltk
nltk.download('punkt')

!pip install pymorphy2

import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk.corpus import stopwords

import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 8.5 MB/s 


In [ ]:
def download_json(url): # запрос на сервер hh.ru
  response = urlopen(url)
  data_json = json.loads(response.read())
  return data_json
  
# download_json('https://api.hh.ru/vacancies/220')['description'] # пример работы 

In [ ]:
'''
РЕАЛИЗОВАТЬ remove_garbage
Используйте регулярные выражения
IN: str '<p>ТРЕБОВАНИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...'
OUT: str 'ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...'
'''
def remove_garbage(raw_text):
  raw_text = re.sub(r'<[^>]*>', '', raw_text)
  raw_text = re.sub(r'\\.', '', raw_text)
  raw_text = re.sub(r'&quot', '', raw_text)

  raw_text = re.sub(r'\s+', ' ', raw_text)
  return raw_text

# remove_garbage('Требования к кандидатам:\r<br />\nВозраст 25-40 лет,\r<br />\nвысшее образование психологическое или экономическое. Предпочтительно два высших образования: \r<br />\n1-е экономическое/техническое, \r<br />\n2-е психологическое (возможно неоконченное). \r<br />\nОпыт реализации или участия в проектах, связанных с постановкой или совершенствованием систем управления персоналом, как внутри организации, так и в качестве привлеченного консультанта от 2-х лет. Полноценный набор знаний и навыков в области управления персоналом (оценка, обучение и развитие, компенсация, мотивация, подбор).\r<br />\nЖелательно наличие знаний в следующих областях: анализ организационной структуры предприятия и  структуры аппарата управления; анализ управленческих регламентов и процедур; управление проектом; организация и принципы функционирования основных управленческих систем. \r<br />\nГибкость мышления, способность к написанию и структурированию текстов, способность видеть и строить механизмы.\r<br />\nГотовность к командировкам (50% рабочего времени),\r<br />\nгражданство РФ. \r<br />\n\r<br />\nПерспективы роста.')

In [ ]:
'''
РЕАЛИЗОВАТЬ tokenize
Используйте word_tokenize из nltk.tokenize
IN: str 'ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...'
OUT: list ['требования', ':', '-', 'обязательно', 'высшее', 'или', 'незаконченное', 'высшее', 'экономическое', 'образование']
'''
def tokenize(raw_text):
  tokenizer = nt.RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(raw_text)
  return tokens

# tokenize('ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...')

In [ ]:
'''
РЕАЛИЗОВАТЬ to_base_form
Можно использовать и лемматизацию и стемминг: pymystem3, ntlk, Spacy, gensim
IN: str 'Красивая мама красиво мыла раму'
OUT: str 'красивый мама красиво мыть рама'
'''
def to_base_form(raw_text):
  res = list()
  for word in raw_text:
    p = morph.parse(word)[0]
    res.append(p.normal_form)
  return res

# to_base_form(['Красивая','мама', 'красиво', 'мыла', 'раму'])

In [ ]:
'''
РЕАЛИЗОВАТЬ remove_stop_words
Используйте stopwords из nltk.corpus 
IN: list ['требования', ':', '-', 'обязательно', 'высший', 'или', 'незаконченный', 'высший', 'экономический', 'образование']
OUT: list ['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование']
'''
nltk.download('stopwords')
stopWords = set(stopwords.words("russian"))

def remove_stop_words(tokens):
  filtered_words = []
  for word in tokens:
    if word not in stopWords:
        filtered_words.append(word)
  return filtered_words

# remove_stop_words(['требования', ':', '-', 'обязательно', 'высший', 'или', 'незаконченный', 'высший', 'экономический', 'образование'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
'''
РЕАЛИЗОВАТЬ get_prepared_vacancy_description_from_hh
Используйте реализованные ранее методы
IN: int 72323
OUT1: str vacancy_json dump
OUT2: list[list] [['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование', ...], [...], ...]
'''
def get_prepared_vacancy_from_hh(vacancy_id):
  base_url = 'https://api.hh.ru/vacancies/'
  vacancy_json = download_json(base_url + str(vacancy_id))
  vacancy_description = vacancy_json['description']

  vacancy_description_sentences = nt.sent_tokenize(vacancy_description)

  prepared_vacancy_description = []
  for sentence in vacancy_description_sentences:
    remove_garbage_sentence = remove_garbage(raw_text=sentence)
    tokenize_sentence = tokenize(remove_garbage_sentence)
    to_base_form_sentence = to_base_form(tokenize_sentence)
    remove_stop_words_sentence = remove_stop_words(to_base_form_sentence)
    prepared_vacancy_description.append(remove_stop_words_sentence)
  return vacancy_id, vacancy_json, prepared_vacancy_description


In [ ]:
'''
РЕАЛИЗОВАТЬ get_prepared_all_vacancies_from_hh
Используйте get_prepared_vacancy_from_hh и ThreadPoolExecutor из concurrent.futures
IN: list [int]
OUT: list [vacancy_id, raw_description, prepared_description]
'''
def get_prepared_all_vacancies_from_hh(df, vacancy_ids):
  with cf.ThreadPoolExecutor() as executor:
    futures = []
    for vacancy_id in vacancy_ids:
      futures.append(executor.submit(get_prepared_vacancy_from_hh, vacancy_id=vacancy_id))
    for future in cf.as_completed(futures):
      try:
        id, json, prepared_description = future.result()
        row = pd.Series({'vacancy_id': id , 'raw_description': json , 'prepared_description': prepared_description})
        df = df.append(row, ignore_index=True)
      except Exception as exc:
        print(f'сгенерировано исключение: {exc}')
  return df


In [47]:
def generation_vacancy_ids(id_start = random.randint(1, 1000000)):
  size = 2000
  test_size = 10
  vacancy_ids = [id_start+id for id in range(size)]
  return vacancy_ids


In [ ]:
# vacancies_df = pd.DataFrame(columns=['vacancy_id', 'raw_description', 'prepared_description'])

In [41]:
vacancies_df = pd.DataFrame
vacancies_df = pd.read_csv('../data/prepared_description_vacancies_id_last_810321_size_8086.csv')
del vacancies_df['Unnamed: 0']
vacancies_df.head()

,vacancy_id,raw_description,prepared_description
0,795348,"{'id': '795348', 'premium': False, 'billing_ty...","[['сфера', 'деятельность', 'розничный', 'торго..."
1,795347,"{'id': '795347', 'premium': False, 'billing_ty...","[['corporate', 'western', 'bank', 'initiates',..."
2,795353,"{'id': '795353', 'premium': False, 'billing_ty...","[['обязанность', 'осуществление', 'качественны..."
3,795350,"{'id': '795350', 'premium': False, 'billing_ty...","[['основной', 'обязанность', 'ведение', 'кадро..."
4,795352,"{'id': '795352', 'premium': False, 'billing_ty...","[['обязанность', 'приём', 'распределение', 'вх..."


In [ ]:
vacancies_df = get_prepared_all_vacancies_from_hh(vacancies_df, generation_vacancy_ids(812321))

In [54]:
vacancies_df.to_csv('../data/prepared_description_vacancies.csv')

In [53]:
vacancies_df['vacancy_id']

0        795348
1        795347
2        795353
3        795350
4        795352
          ...  
10108    814105
10109    814108
10110    814113
10111    814119
10112    814120
Name: vacancy_id, Length: 10113, dtype: int64

In [52]:
vacancies_df.shape

(10113, 3)